# Sorted Set examples

## Connect to Redis

To understand what ```decode_responses=True``` does, refer back to [this document](connection_examples.ipynb)

In [ ]:
import redis

r = redis.Redis(decode_responses=True)
r.ping()

## Basic ```ZADD``` and ```ZRANGE```

```ZADD``` adds members with scores to a sorted set. Members are automatically ordered by score.

In [ ]:
r.delete("game:scores")

r.zadd("game:scores", {"alice": 100, "bob": 200, "charlie": 150})

```ZRANGE``` returns members ordered by score (ascending by default)

In [ ]:
r.zrange("game:scores", 0, -1)

Pass ```withscores=True``` to include scores in the result

In [ ]:
r.zrange("game:scores", 0, -1, withscores=True)

## ```ZRANGEBYSCORE```

Query members whose scores fall within a given range

In [ ]:
r.zrangebyscore("game:scores", 100, 150, withscores=True)

Use ```"-inf"``` and ```"+inf"``` for unbounded ranges

In [ ]:
r.zrangebyscore("game:scores", "-inf", 150, withscores=True)

## ```ZINCRBY```

Increment the score of a member. This is useful for updating scores in real time.

In [ ]:
r.zincrby("game:scores", 75, "alice")

In [ ]:
r.zrange("game:scores", 0, -1, withscores=True)

## ```ZRANK``` and ```ZREVRANK```

```ZRANK``` returns the rank of a member (0-based, lowest score first). ```ZREVRANK``` returns the rank with the highest score first.

In [ ]:
r.zrank("game:scores", "bob")

In [ ]:
r.zrevrank("game:scores", "bob")

## ```ZREM```

Remove one or more members from a sorted set

In [ ]:
r.zrem("game:scores", "charlie")

In [ ]:
r.zrange("game:scores", 0, -1, withscores=True)

## ```ZUNIONSTORE``` and ```ZINTERSTORE```

Combine multiple sorted sets. ```ZUNIONSTORE``` stores the union and ```ZINTERSTORE``` stores the intersection of the input sets.

In [ ]:
r.delete("game:round1", "game:round2", "game:total", "game:common")

r.zadd("game:round1", {"alice": 100, "bob": 200})
r.zadd("game:round2", {"bob": 150, "charlie": 300})

Union sums scores by default for members that appear in both sets

In [ ]:
r.zunionstore("game:total", ["game:round1", "game:round2"])
r.zrange("game:total", 0, -1, withscores=True)

Intersection keeps only members present in all input sets

In [ ]:
r.zinterstore("game:common", ["game:round1", "game:round2"])
r.zrange("game:common", 0, -1, withscores=True)

## Practical example: Leaderboard

Sorted sets are a natural fit for leaderboards. Scores are kept in order automatically, making rank lookups very efficient.

In [ ]:
r.delete("leaderboard")

# Add players with their scores
r.zadd(
    "leaderboard",
    {
        "player:anna": 2500,
        "player:ben": 4000,
        "player:cara": 3500,
        "player:dan": 1800,
        "player:eve": 4200,
    },
)

In [ ]:
# Top 3 players (highest scores first)
r.zrevrange("leaderboard", 0, 2, withscores=True)

In [ ]:
# A player completes a level and earns bonus points
r.zincrby("leaderboard", 600, "player:anna")

# Check anna's new rank (0-based, highest score first)
r.zrevrank("leaderboard", "player:anna")

In [ ]:
# Get a specific player's score
r.zscore("leaderboard", "player:anna")

In [ ]:
# Total number of players on the leaderboard
r.zcard("leaderboard")

In [ ]:
# Final leaderboard (highest to lowest)
r.zrevrange("leaderboard", 0, -1, withscores=True)